In [ ]:
import pandas as pd
import numpy as np
from random import randint
import time
import re
# Visualizations
import matplotlib.pyplot as plt
import seaborn as sn
# Scraping through Chrome webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
# Starting URLs
centris = "https://www.centris.ca/en/properties~for-sale?view=Thumbnail"
duproprio = "https://duproprio.com/en/search/list?search=true&is_for_sale=1&with_builders=1&parent=1&pageNumber=1&sort=-published_at"

In [ ]:
class Centris:
    """
    This class represents a chrome webdriver object with access to centris.ca.
    
    Attr:
    self.url - starting url for scraping process
    self.data - pandas.DataFrame object containing scraped data
    self.driver - Chrome webdriver
    self.containers - List of web-elements containing information on listings
        - eg: link to listing, price, picture, address,...
    self.links_to_listings - List of web-elements, each containing the link to a listing
    """
     
    def __init__(self, url="https://www.centris.ca/en/houses~for-sale~lac-simon/11851081?view=Summary&uc=3"): 
        self.url = url
        self.data = pd.DataFrame()
        # Path to Chromedriver
        self.DRIVER_PATH = 'C:/webdriver/chromedriver.exe'
        self.driver = None
        # Verification for new DOM
        self.old_DOM = {\
                        'title' : [],\
                        'address' : [],\
                        'price' : [],\
                        'lat' : [],\
                        'long' : [],\
                        'descriptions' : [],\
                        'neighbourhood_top' : [],\
                        'neighbourhood_mid' : [],\
                        'neighbourhood_buttom' : [],\
                        'demographics_buttons' : [],\
                    }
        
    def reset_old_DOM(self):
        self.old_DOM = {\
                        'title' : [],\
                        'address' : [],\
                        'price' : [],\
                        'lat' : [],\
                        'long' : [],\
                        'descriptions' : [],\
                        'neighbourhood_top' : [],\
                        'neighbourhood_mid' : [],\
                        'neighbourhood_buttom' : [],\
                        'demographics_buttons' : [],\
                    }

    def append_data(self, title, address, price,\
            lat, long, descriptions, neighbourhood_indicators,\
            population, demographics):
        """Appends new data to existing data frame.
        
        Args:
        title - string
        address - string 
        price - 
        lat - 
        long - 
        descriptions - 
        neighbourhood_indicators -
        population - 
        demographics - 
        """
        new_data = pd.DataFrame({\
                        'title': title,\
                        'address': address,\
                        'price': price,\
                        'lat': lat,\
                        'long': long\
                    }, index=[0])
        
        # DESCRIPTIONS
        description_table = pd.DataFrame()
#         headers_of_interest = [\
#                 "rooms", "bedrooms", "powder room", "Number of units", "Building style",\
#                 "Condominium type", "Year built", "Building area", "Lot area", "walk_score",\
#                 "Net area", "Parking (total)", "Main unit", "Potential gross revenue", "Pool"\
#                               ]
#         # Ensures consistency accross listings
#         for header in headers_of_interest:
#             if header in descriptions.keys():
#                 value = descriptions[header]
#             else:
#                 value = np.nan
#             description_table[header] = pd.Series(value)

        for key in descriptions.keys():
            header = key
            value = descriptions[header]
            description_table[header] = pd.Series(value)
        
        # POPULATION AND DEMOGRAPHICS
        new_data = pd.concat([new_data, neighbourhood_indicators, description_table,\
                             population, demographics], axis=1)
        # LOGGING --------------------------     
        #print(new_data)
        
        self.data = self.data.append(new_data, sort=False,\
                                     ignore_index=True)
        
    
    def get_data(self):
        return self.data
        
    def start_driver(self):
        """
        Starts and returns Crome webdriver. 
        The page link in the url attribute 
        is opened in headless mode.
        """
        
        # Activate headless mode for fastest response
        options = Options()
        options.add_argument("--headless")
        options.add_argument("--disable-infobars"); # disabling infobars
        options.add_argument("--disable-extensions"); # disabling extensions
        options.add_argument("--disable-gpu"); # applicable to windows os only
        options.add_argument("--disable-dev-shm-usage"); # overcome limited resource problems
        options.add_argument("--no-sandbox"); # Bypass OS security model
        options.add_argument('--start-maximized') # open Browser in maximized mode
        options.add_argument('--incognito')

        # Start driver with url
        self.driver = webdriver.Chrome(executable_path=self.DRIVER_PATH)
        self.driver.get(self.url)

    def sort_listings(self):
        """Sorts listings in webdriver from newest to oldest."""
        
        # Click drop down menu
        drop_down = self.driver.find_element_by_xpath(\
                                    "//button[@id='dropdownSort']")
        drop_down.click()
        
        # Sort by most recent listings
        sort_by = self.driver.find_element_by_xpath("//a[@data-option-value='3']")
        sort_by.click()
    
    def goto_first_page(self):
        try:
            next_page = self.driver.find_element_by_xpath(\
                                        "//li[@class='goFirst']")
            next_page.click()
        except:
            print("goFirst button not available")
    
    def next_page(self):
        try:
            next_page = self.driver.find_element_by_xpath(\
                                        "//li[@class='next']")
            next_page.click()
            pass
        except:
            time.sleep(0.5)
            # Try again after waiting 0.5 sec.
            try:
                next_page = self.driver.find_element_by_xpath(\
                                            "//li[@class='next']")
                next_page.click()
                pass
            except:
                print("Next-page button not found!")
                
    def get_page_position(self):
        '''Returns the first and last page of the current search.
        
        Returns
        tuple - (current_page, last_page), '''
        
        pages = self.driver.find_element_by_xpath(\
                                    "//li[@class='pager-current']").text.\
                                    split(" / ")
        
        current_page, last_page = (int(page.replace(",","")) for page in pages)
        
        return (current_page, last_page)
    
    def refresh_page(self):
        "Refreshes current webdriver page."
        self.driver.refresh()
        print("Page is being refreshed.")
        # Wait until page fully loaded
        time.sleep(2)
        
    def distance(origin, destination):
        """Calculates distances from latitudinal/longitudinal data using
        the haversine formula"""
        lat1, lon1 = origin
        lat2, lon2 = destination
        radius = 6371 # km
        
        #Convert from degrees to radians
        dlat = math.radians(lat2-lat1)
        dlon = math.radians(lon2-lon1)
        
        # Haversine formula
        a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
            * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        d = radius * c

        return d
                
                                                 
# Instantiate class object
centris = Centris()

The following functions need to be outside of the Class. wait_for_xpath() determined the approptiate time to call get_data(). Initially, both fuctions were part of the class object. It seems that after the get_data() call, the driver does not get updated within the class. This leads in some cases to old DOM's being accessed after the browser has already switched to the next page. To circumvent this issue, elements are called outside the class and tried until accessible. This allows the entire new DOM to be loaded before get_data() is called.

In [ ]:
 def wait_for_xpath(xpath: str, old_element):
        """
        Wait until elements in new DOM are accessible.
        
        Arg.
        xpath - xpath to new element 
        old_element - element at xpath from previous DOM (found in centris.old_DOM)
        
        Returns:
        current_element - the element found in the new DOM at xpath
        """
        
        centris_driver = centris.driver
        element_at_xpath = []
        
        # Ensure that the NEW rather than the previous or no DOM is active
        # Maximum wait time 10 sec.
        time_passed = 0
        while (\
            (element_at_xpath == old_element or  element_at_xpath == [])\
            and (time_passed <= 10)\
              ):
            # Wait for DOM to load
            time.sleep(0.2)
            time_passed += 0.2
            
            # Print every 2 seconds
            if time_passed%2 == 0:
                print("Waiting for new DOM...")
            
            # Attempt to load new DOM
            try: 
                element_at_xpath = centris_driver.find_elements_by_xpath(xpath)
            except: pass
        
        # After 10 seconds unlikely to load at all -> restart entire process
        if time_passed > 10:
            print("RuntimeError: element not found.")
            centris.refresh_page()
            get_data_from_centris()
            wait_for_xpath(xpath, old_element)
            
        return element_at_xpath

In [ ]:
def scrape_description(old_DOM):
    """ Requires instantiated centris object. Scrapes and returns
    description data: Year build, price, Net area, etc."""
    
    descriptions = wait_for_xpath("//div[@class='col-lg-12 description']",\
                                 old_DOM)
    #First three elements not relevant
    descriptions_list = descriptions[0].text.split("\n")[3:]
    
    #LOGGING------------------------
    #print("DESCRIPTION:", descriptions_list)
    
    # Update old_DOM dictionary with new element for next verification
    centris.old_DOM['descriptions'] = descriptions
    
    return extract_descriptions(descriptions_list)

In [ ]:
def extract_descriptions(descriptions_list):
    """Takes in data from scrape_description() and returns it 
    as a dictionary"""
    
    # The data_dict found on this part of the page is inconsistent across listings
    # The first row may contain the number of rooms, bedrooms and bathrooms without headers or may be missing
    # Following rows have heathers with associated values after a line break
    # The very last element may be a walking score without header
    # Listings without first row may supply first row information in subsequent rows with headers
    # Because of these inconsistencies, two seperate extractions need to be implemented: one for
    # first row lements (if they exist) and another for subsequent rows
    
    # Transformed data
    data_dict = {}
    # Distinguish between elements from first and subsequent rows if first row exists
    first_row = True
    # Starting point for second part of transformation
    second_row_index = 0
    
    # First Part
    while first_row == True:
        for description in descriptions_list:
            numeric = re.findall("\(*[0-9]+\)*", description) # numbers
            text = re.findall("[A-Za-z]+[A-Za-z\s\-]*", description) # text after/inbetween numbers 

            # Initial elements with numeric values correspond to first row
            if (numeric != []):
                # For each value there must be one text description
                if (len(numeric) == len(text)):
                    for description,value in zip(text, numeric):
                        # Save as column in data_dict
                        description_clean = description.replace("and", "").strip()
                        data_dict[description_clean] = value
                    second_row_index += 1 
                else:
                    print("Unequal number of first row keys and values!")
                    print("Numbers:", numeric)
                    print("Text:", text)
                    break
            else:
                first_row = False # No numeric information implies header
                break
    
    # Index range of second extraction
    # Headers are found at every second index (0,2,4,...)
    # Values are one index apart from their corresponding header (1,3,5,...)
    list_length = len(descriptions_list)
    if (list_length - second_row_index)%2 == 1: # Implies presence of element without header -> Walk Score
        walk_score_listed = True
        end_point = list_length -1
    else:
        walk_score_listed = False
        end_point = list_length
    # Indices corresponding to headers
    extraction_range = range(second_row_index, end_point, 2)
    
    #LOGGING----------------------
#     print("Second row index:", second_row_index)
#     print("Extraction range:", extraction_range)
#     print("List:", descriptions_list)
    
    # Second Part
    for header_index in extraction_range:
        # Headers as column names
        header = descriptions_list[header_index]
        # Values corresponding to headers are found at subsequent indices
        information = descriptions_list[header_index + 1] 
        data_dict[header] = information
    
    if walk_score_listed:
        data_dict["walk_score"] = descriptions_list[-1]
        #LOGGING----------------------
        #print("Walk Score:", descriptions_list[-1])
        
    #LOGGING--------------------------
#     print("Descriptions:", data_dict)
        
    return data_dict

In [ ]:
def scrape_neighbourhood(old_DOM_top, old_DOM_mid, old_DOM_buttom):
    """ Scrapes and returns a list of ratings 
    between 0-10 for a set of neighborhood indicators
    such as groceries, parks, noise, etc.)
    """
    driver = centris.driver
    
    # Extract elements from top section of scrollable list
    neighbourhood_top = wait_for_xpath(\
                            "//div[@class='ll-list ps ps--active-y']",\
                            old_DOM_top)
    # Split into indicators and ranking values
    top = [x.text for x in neighbourhood_top][0].split("\n")
    
    # LOGGING----------------------
#     print("Top neighbourhood:", top)
    
    # Extract middle section - only one element
    # Scroll and activate scrollable bar container
    scrollable_bar = driver.find_element_by_xpath(\
                                            "//div[@class='ps__thumb-y']")
    ActionChains(driver).\
        move_to_element(scrollable_bar).\
        send_keys(Keys.PAGE_DOWN).\
        click(scrollable_bar).perform()

    # Elements from buttom of scrollable list
    neighbourhood_mid = wait_for_xpath(\
                            "//div[@class='ll-list ps ps--active-y']",\
                            old_DOM_mid)
    # Split into indicators and ranking values
    
    # LOGGING----------------------
    #print("Neighbourhoud mid section:", neighbourhood_mid)
    
    middle = [x.text for x in neighbourhood_mid][0].split("\n")
    
    # Extract buttom section
    # Scroll and load remaining elements
#     scrollable_bar = driver.find_element_by_xpath(\
#                                             "//div[@class='ps__thumb-y']")
    ActionChains(driver).\
        move_to_element(scrollable_bar).\
        send_keys(Keys.PAGE_DOWN).\
        click(scrollable_bar).perform()
    
    # Elements from buttom of scrollable list
    neighbourhood_buttom = wait_for_xpath(\
                            "//div[@class='ll-list ps ps--active-y']",\
                            old_DOM_buttom)
    # Split into indicators and ranking values
    buttom = [x.text for x in neighbourhood_buttom][0].split("\n")
    
    # LOGGING----------------------
#     print("Buttom neighbourhood:", buttom)
    
    # Unite all three sections by storing tuples of indicator names and corresponding values
    united_list = []
    list_length = len(top)
    for i in range(0, list_length, 2):
        united_list.append((top[i], top[i+1]))
        united_list.append((middle[i], middle[i+1]))
        united_list.append((buttom[i], buttom[i+1]))
    
    # Create set of unique tuples
    neighbourhood_indicators = set(united_list)
    
    # LOGGING----------------------
#     print("Number of neighborhood indicators: ", len(neighbourhood_indicators))
#     print("UNITED:", united_list)
#     print("SET:", neighbourhood_indicators)
    
    # Verify size and extract information as list
    # If size unexpected, refresh page and restart process
    if len(neighbourhood_indicators) < 8:
            centris.refresh_page()
            scrape_neighbourhood(old_DOM_top, old_DOM_buttom)

    # Update old_DOM dictionary with new elements for next verification
    centris.old_DOM['neighbourhood_top'] = neighbourhood_top
    centris.old_DOM['neighbourhood_mid'] = neighbourhood_mid
    centris.old_DOM['neighbourhood_buttom'] = neighbourhood_buttom

    return extract_neighbourhood_indicators(neighbourhood_indicators)

In [ ]:
def extract_neighbourhood_indicators(indicators):
    """Takes in neighbourhood data from scrape_neighbourhood() and returns it 
    in tabular form as a DataFrame object"""
    data = pd.DataFrame()
    for indicator in indicators:
        header = indicator[0]
        value = indicator[1]
        data[header] = pd.Series(value)
    
    return data

In [ ]:
def scrape_population():
    """Scrapes and returns population summary data (density, variation etc.)"""
    population_summaries =  centris.driver.find_element_by_id('info')
    population_summaries_list = population_summaries\
                        .text.split("\n")
    
    # LOGGING-----------------------
    #print("Population:", population_summaries_list)
    
    return extract_population(population_summaries_list)

In [ ]:
def extract_population(population):
    """Takes in population data from scrape_population() and returns it 
    in tabular form as a DataFrame object"""
    
    data = pd.DataFrame()
    for info in population:
        units_removed =  info.replace("hab/km2", "").strip()
        # Numeric data
        numeric = re.findall("[0-9]+[0-9,]*", units_removed)
        numeric_clean = numeric[-1].replace(",","")

        # Text data for column names
        header = re.findall("[a-zA-Z\s]+", units_removed)
        header_clean = header[0]
        # Add numeric data to header excluding the value at index -1
        for numeric_head_data in numeric[:-1]:
            header_clean = header_clean + str(numeric_head_data) + " "

        data[header_clean] = pd.Series(numeric_clean).astype("int")
    return data

In [ ]:
def scrape_demographics(old_DOM):
    """Scrapes and return demographic data found in a clickable list"""
    
    driver = centris.driver
    # Clickable list containing demographic data
    menu = driver.find_element_by_id("menu")
    # Load menu by moving browser to it
    ActionChains(driver).\
    move_to_element(menu).perform()
    
    #Buttons to access demographics data (education, incomes, etc.)
    demographics_buttons = wait_for_xpath(\
                        "//div[@class='centrisSocioDemobutton']",\
                                                 old_DOM)

    # LOGGING------------------------
    # print("DEMO. BUTTONS:", demographics_buttons)

    # First entry on clickable demographics list (pre-selected)
    demographics = []

    # Click buttons to access next demogrpahics elements
    for button in demographics_buttons:
        try: 
            button.click()
        except: 
            print("Demographics button missing!")
            # Reattempt loading buttons
            centris.refresh_page()
            ActionChains(driver).\
            move_to_element(menu).\
            perform()
            time.sleep(2) # extra time to load
            demographics_buttons = wait_for_xpath(\
                        "//div[@class='centrisSocioDemobutton']",\
                                                 old_DOM)
            
        # Get and append data after button click
        demographic_data = driver.find_element_by_class_name(\
                         "socioDemoLabel")
        demographics.append(demographic_data.text)
    
    # Split each demographic component into separate list
    # Example: splits "Occupation" data into -> ["Owners", "35%", "Renters", "65%"]
    demographics = [demo.split("\n") for demo in demographics]
    
    #LOGGING------------------------
#     print("DEMO. DATA:", demographics)
#     print("-"*50)
    
    # Update old_DOM dictionary with new elements for next verification
    centris.old_DOM['demographics_buttons'] = demographics_buttons
    
    return extract_demographics(demographics)

In [ ]:
def extract_demographics(demographics):
    """Takes in demographic data from extract_demographics() and returns it 
    in tabular form as a DataFrame object"""
    
    data = pd.DataFrame()
    
    for demographic in demographics:
        # Remove empty stings from splitting double line breaks \n\n
        removed_empty_strings = [x for x in demographic if x != ""]
        # Format of demographic: [header, value, header, value, ...]
        header_index = range(0, len(demographic), 2)
        for i in header_index:
            header = demographic[i] + " (%)" # add units to column names
            value = demographic[i+1].replace("%", "") # remove units from values 
            data[header] = pd.Series(value).astype("int")

    return data

In [ ]:
def get_data_from_centris():
        """
        Requires instantiate Centris object. Scrapes information from the
        webdriver and appends it to the Centris object.
        """
        driver = centris.driver
        old_DOM = centris.old_DOM
        
        # Data from headers
        print("Start scraping new page...")
        title = wait_for_xpath("//span[@data-id='PageTitle']", old_DOM['title'])
        address = wait_for_xpath("//h2[@itemprop='address']", old_DOM['address'])
        price = wait_for_xpath("//span[@itemprop='price']", old_DOM['price'])
        lat = wait_for_xpath("//meta[@itemprop='latitude']", old_DOM['lat'])
        long = wait_for_xpath("//meta[@itemprop='longitude']", old_DOM['long'])
        
        # Save elements as old DOM
        centris.old_DOM['title'] = title
        centris.old_DOM['address'] = address
        centris.old_DOM['lat'] = lat
        centris.old_DOM['long'] = long
        
        # Scrape remaining elements and store in dataframe
        descriptions = scrape_description(old_DOM['descriptions'])
        neighbourhood_indicators = scrape_neighbourhood(old_DOM['neighbourhood_top'],\
                                                            old_DOM['neighbourhood_mid'],\
                                                            old_DOM['neighbourhood_buttom'])
        population = scrape_population()
        demographics = scrape_demographics(old_DOM['demographics_buttons'])
                
        # Unify data in single dataframe and append to results table
        centris.append_data(
            title[0].text,\
            address[0].text,\
            price[0].text,\
            lat[0].get_attribute("content"),\
            long[0].get_attribute("content"),\
            descriptions,\
            neighbourhood_indicators,\
            population,\
            demographics\
        )
        
        # LOGGING--------------------------
        #print("GET DATA: DESCRIPTIONS:", descriptions)
        
        # Return to top of page, to access next-page button
        body = driver.find_element_by_tag_name("body")
        body.send_keys(Keys.HOME)
#         for i in range(7):
#             body.send_keys(Keys.PAGE_UP)

## Initiate scraping

In [ ]:
# Test
centris = Centris()
start = time.time()
centris.start_driver()
centris.sort_listings()
print("Execution time:", time.time() - start)

Before running the next cell, search for the region(s) you want to scrape in the webdriver window.
This is not required but will narrow results and reduce runtime.

In [ ]:
start = time.time() 
current_page, last_page = centris.get_page_position() 
pages_to_scrape = last_page - current_page + 1 # in case scraping is interupted
one_to_100 = range(1,100) # to print message each 1% completion

print("Scraping initiated.")
print("Total number of pages to scrape:", pages_to_scrape)
print("Estimated runtime:", round(pages_to_scrape*((9.6)/(60*60)), 2), "hours")
print("="*50)

for i in range(pages_to_scrape):
    
    print("="*50)
    print("Page:", i+1)
    time_passed = 0 # to exit while loop after 10 seconds
    
    #Refresh every 20 pages to clear memory build-up
    if (i+1)%20 == 0:
        print("Clearing memory")
        print("-"*50)
        
        # Each refresh frees some memory. Four seem to work best.
        for i in range(4):
            centris.refresh_page()
            # Extra time for last refresh
            # Ensures that DOM is fully loaded
            if i == 3:
                time.sleep(2)
            
    #Retrieve data    
    get_data_from_centris()
    
    # Short delay for chrome to respond to PAGE_UP command
    time.sleep(0.5)
    centris.next_page()
    
    # Percent completed of scraping 
    percent_complete = round(100*((i)/total_pages),2)      
    # Print after every 1% mark
    if percent_complete in one_to_100:
        execution_time = (time.time() - start)/(i+1) # seconds per page
        print(percent_complete, "%", "completed")
        print("Average execution time per page:", round(execution_time, 2), "sec.")
        print("Estimated remaining runtime:", round(\
                                (total_pages - (i+1))\
                                *(execution_time\
                                /(60*60)), 1\
                                                   ), "hours <", "-"*50)
        print("="*50)


print("Total runtime:", execution_time/(60*60), "hours")
centris.data

In [ ]:
centris.data.to_csv("centris_montreal_complete.csv")

# Data wrangling

In [723]:
df = pd.read_csv("data_tables/centris_montreal_complete.csv")

In [724]:
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5732 entries, 0 to 5731
Data columns (total 101 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Unnamed: 0                               5732 non-null   int64  
 1   title                                    5732 non-null   object 
 2   address                                  5732 non-null   object 
 3   price                                    5732 non-null   object 
 4   lat                                      5732 non-null   float64
 5   long                                     5732 non-null   float64
 6   Restaurants                              5724 non-null   float64
 7   Shopping                                 5726 non-null   float64
 8   Vibrant                                  5724 non-null   float64
 9   Cycling friendly                         5723 non-null   float64
 10  Car friendly                             5721 n

We first remove the last 5 columns since they have virtually no content

In [725]:
df = df.loc[:,:"bedroom in basement"]

Next we will change the data type of the price column to integer

In [726]:
price_clean = df.price.str.replace("[$,]", "")
df.price = price_clean.astype("int")
df.price.describe()

count    5.732000e+03
mean     7.634336e+05
std      8.866054e+05
min      5.000000e+03
25%      3.999000e+05
50%      5.590000e+05
75%      8.242250e+05
max      2.780000e+07
Name: price, dtype: float64

## Duplicate data

We will use the `address` column to check for duplications.

In [727]:
df.address.duplicated().sum()

308

In [728]:
df = df[df.address.duplicated() == False]
df.reset_index(inplace=True)
df.info(max_cols=95)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5424 entries, 0 to 5423
Columns: 97 entries, index to bedroom in basement
dtypes: float64(32), int64(41), object(24)
memory usage: 4.0+ MB


Removing duplicates leaves holes in the index. Due to `reset_index()`, the range now coincides with the number of entries. 
The first two columns, corresponding to copies of the old and new index, can be removed.

In [729]:
df = df.loc[:, "title":]
df.columns

Index(['title', 'address', 'price', 'lat', 'long', 'Restaurants', 'Shopping',
       'Vibrant', 'Cycling friendly', 'Car friendly', 'Historic', 'Quiet',
       'Elementary Schools', 'High Schools', 'Parks', 'Nightlife', 'Groceries',
       'Daycares', 'Pedestrian friendly', 'Cafes', 'Transit friendly',
       'Greenery', 'rooms', 'bedrooms', 'bathroom', 'Condominium type',
       'Net area', 'Year built', 'Parking (total)', 'Population 2016 ',
       'Population variation between 2011 2016 ', 'Population density ',
       'Unemployment rate 2016 ', 'Less than $50,000 (%)',
       'Between $50,000 and $80,000 (%)', 'Between $80,000 and $100,000 (%)',
       'Between $100,000 and $150,000 (%)', 'More than $150,000 (%)',
       '1-person households (%)', '2-person households (%)',
       '3-person households (%)', '4-person households (%)',
       '5-person or more households (%)',
       'Couples without children at home (%)',
       'Couples with children at home (%)', 'Single-parent fa

## Merge Columns

There are two columns called `rooms` and `room`. We will inspect both to find an explenation for this.

In [730]:
df[df.room.notnull()].loc[:,["address", "room", "rooms"]].head()

,address,room,rooms
32,"4225, Rue Saint-Dominique, apt. 209, Montréal ...",1.0,NaN
95,"318Z - 330Z, Rue Sherbrooke Est, Montréal (Vil...",1.0,NaN
109,"1188, Rue Saint-Antoine Ouest, apt. P3-16-17, ...",1.0,NaN
142,"1000, Rue Ottawa, apt. 606, Montréal (Le Sud-O...",1.0,NaN
283,"6541, boulevard Saint-Laurent, apt. 201, Montr...",1.0,NaN


The room column corresponds to listings with with only a single room. This has been verified by visting the website and searching for some of the addresses. We will therefore merge the two columns.

In [731]:

print("Non-null values in 'rooms':")
print("Before merge -", df.rooms.notnull().sum())
# Merging "rooms" and "room"
rooms_new = pd.Series(\
    [room if pd.notna(room) else rooms for room, rooms in zip(df.room, df.rooms)]\
        )
# Remove old columns
df.drop(["rooms", "room"], axis=1, inplace=True)
# Add new column
df["rooms"] = rooms_new
print("After merge -", df.rooms.notnull().sum())

Non-null values in 'rooms':
Before merge - 4421
After merge - 4451


We will now incpect other columns that contain the string "room" and replace missing values by 0 to facilitate mathematical operations on room data. 

In [732]:
rooms = df.columns[["room" in col for col in df.columns]]
df[rooms] = df[rooms].fillna(0)
print(df[rooms].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5424 entries, 0 to 5423
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bedrooms              5424 non-null   float64
 1   bathroom              5424 non-null   float64
 2   bathrooms             5424 non-null   float64
 3   bedroom               5424 non-null   float64
 4   powder room           5424 non-null   float64
 5   powder rooms          5424 non-null   float64
 6   bedrooms in basement  5424 non-null   float64
 7   bedroom in basement   5424 non-null   float64
 8   rooms                 5424 non-null   float64
dtypes: float64(9)
memory usage: 381.5 KB
None


In [733]:
# Grouping similar columns
bedroom_cols = ['bedrooms','bedroom','bedrooms in basement', 'bedroom in basement']
bathroom_cols = ['bathroom', 'bathrooms']
powederroom_cols = ['powder room', 'powder rooms']

### Bedrooms

In [734]:
df[bedroom_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5424 entries, 0 to 5423
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bedrooms              5424 non-null   float64
 1   bedroom               5424 non-null   float64
 2   bedrooms in basement  5424 non-null   float64
 3   bedroom in basement   5424 non-null   float64
dtypes: float64(4)
memory usage: 169.6 KB


First, we merge `bedrooms` and `bedroom` since they clearly belong together

In [735]:
# Replace missing values with 0 for element-wise addition
bedrooms = df.bedrooms.fillna(0) + df.bedroom.fillna(0)
# Drop old columns from df
df.drop(["bedrooms", "bedroom"], axis=1)
# Add new column
df.bedrooms = bedrooms

In [736]:
# At least 1 bedroom in basement
df[df["bedrooms in basement"] > 0].\
    loc[:, ["address", "bedrooms", "bedrooms in basement", "bedroom in basement", "price"]].head()

,address,bedrooms,bedrooms in basement,bedroom in basement,price
151,"61A, Avenue Strathyre, Montréal (LaSalle)",0.0,2.0,0.0,259000
273,"1300, Rue Jarry Est, apt. 1, Montréal (Villera...",0.0,2.0,0.0,259000
353,"2061, Rue Beaudry, Montréal (Ville-Marie), Nei...",0.0,2.0,0.0,419000
627,"4723, Rue Ontario Est, apt. 102, Montréal (Mer...",0.0,2.0,0.0,259000
670,"7940, Rue Chabot, apt. 102, Montréal (Villeray...",0.0,2.0,0.0,379000


Bedrooms in the basement may be worth less, merging them with regular bedrooms may therefore not be optimal. However, the listings with bedrooms in the basement don't have regular bedrooms. This is true for for the entire list as well as for listings with a single bedroom in the basement. Merging all columns, therefore, appears to be the best solution. To keep a record of basement bedrooms, we will include a boolean column that can be used to either remove or alter corresponding entries at a later point.

In [737]:
def not_zero(col):
    """Returns boolean list indicating records with counts of at least 1"""

    not_zero = [count > 0 for count in col]
    return not_zero

In [738]:
# Merge of basement bedroom columns
basement_bed = df["bedrooms in basement"]\
         + df["bedroom in basement"]

# Records with basement bedrooms
basement_bed_bool = not_zero(basement_bed)
print("Apartments with basement bedrooms:",\
    sum(basement_bed_bool))

# Merge of basement with regular bedrooms
all_bedrooms = basement_bed + df.bedrooms
print("Apartments with any kind of bedroom",\
    sum(not_zero(all_bedrooms)))

Apartments with basement bedrooms: 75
Apartments with any kind of bedroom 4402


In [739]:
# Remove old columns 
df.drop(bedroom_cols, axis=1, inplace=True)

# Append new columns
df["bedrooms"] = all_bedrooms
df["basement_bedroom"] = basement_bed_bool

## Bathrooms

In [740]:
# Non-zero records
df[bathroom_cols].apply(lambda col: sum(not_zero(col)), axis=0)

bathroom     2465
bathrooms    1967
dtype: int64

In [741]:
# Merge
bathrooms = df.bathroom + df.bathrooms
print("Non-zero records of merged column:", sum(not_zero(bathrooms)))

# Remove old columns 
df.drop(bathroom_cols, axis=1, inplace=True)

# Append new column
df["bathrooms"] = bathrooms

Non-zero records of merged column: 4432


In [742]:
df.iloc[:, -4:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5424 entries, 0 to 5423
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rooms             5424 non-null   float64
 1   bedrooms          5424 non-null   float64
 2   basement_bedroom  5424 non-null   bool   
 3   bathrooms         5424 non-null   float64
dtypes: bool(1), float64(3)
memory usage: 132.5 KB


## Powder rooms

In [743]:
# Non-zero records
df[powederroom_cols].apply(lambda col: sum(not_zero(col)))

powder room     1086
powder rooms      81
dtype: int64

In [744]:
# Merge
powder_rooms = df['powder room'] + df['powder rooms']

# Remove old 
df.drop(powederroom_cols, axis=1, inplace=True)

# Append new
df['powder_rooms'] = powder_rooms

In [745]:
# New room data
room_data = df.iloc[:, [-5,-4,-2,-1]]

print("Non-zero records:")
room_data.apply(lambda col: sum(not_zero(col)))

Non-zero records:


rooms           4451
bedrooms        4402
bathrooms       4432
powder_rooms    1167
dtype: int64

## Missing Room Data

Next we will inspect records with missing data for all 4 room columns

In [746]:
# Records with 0 for all room features
# "not any(not_zero(rows))" is True, only if all records in a row are zero
missing_room_data = df[room_data.apply(lambda rows: (not any(not_zero(rows))), axis=1)]

print("Number of records with missing room data:", \
    missing_room_data.shape[0])
print("Number of replacable missing room data:", \
    missing_room_data.loc[:, "Main unit"].notnull().sum())

missing_room_data.iloc[:,[0, 1, 2,-5,-4,-2, -1]].head()

Number of records with missing room data: 973
Number of replacable missing room data: 868


,title,address,price,rooms,bedrooms,bathrooms,powder_rooms
9,Duplex for sale,"5421 - 5423, 15e Avenue, Montréal (Rosemont/La...",679000,0.0,0.0,0.0,0.0
10,Quadruplex for sale,"9636 - 9640, Rue Notre-Dame Est, Montréal (Mer...",569750,0.0,0.0,0.0,0.0
25,Duplex for sale,"1915 - 1917, Rue Viau, Montréal (Mercier/Hoche...",899000,0.0,0.0,0.0,0.0
29,Triplex for sale,"5518 - 5522, boulevard Saint-Michel, Montréal ...",719700,0.0,0.0,0.0,0.0
34,Triplex for sale,"6426 - 6428, Rue Marie-Victorin, Montréal (Mon...",599000,0.0,0.0,0.0,0.0


There are quite a lot of records with no room data in either of the 4 new columns that we just generated. 
The `Main unit` column is able to fill in most of the missing data.

In [747]:
missing_room_data["Main unit"].head()

9     6 rooms, 3 bedrooms, 2 bathrooms and 1 powder ...
10                       3 rooms, 1 bedroom, 1 bathroom
25                    11 rooms, 3 bedrooms, 2 bathrooms
29                      5 rooms, 2 bedrooms, 1 bathroom
34                      5 rooms, 3 bedrooms, 1 bathroom
Name: Main unit, dtype: object

From the above displayed records it seems most appropriate to split according to `","` and `"and"`

In [748]:
# Split room data in "Main unit"
main_split = df["Main unit"].str.split(", | and")
main_split

0                                     NaN
1                                     NaN
2                                     NaN
3                                     NaN
4                                     NaN
                      ...                
5419                                  NaN
5420                                  NaN
5421                                  NaN
5422    [4 rooms, 2 bedrooms, 1 bathroom]
5423                                  NaN
Name: Main unit, Length: 5424, dtype: object

In [749]:
def extract_rooms(room_list):
    """Turns the list into a DataFrame object, with text
    as column names and numbers as column values"""
    
    # Expected room types
    room_types = ["rooms", "room",\
        "bedrooms", "bedroom",\
        "bathrooms", "bathroom",\
        "powder rooms", "powder room"]

    # No room data
    if room_list is np.nan:
        return pd.Series([0,0,0,0,0,0,0,0],\
            index=room_types)

    # Result table   
    room_data = pd.Series()

    # Extract and append room data
    for string in room_list:
        number_of_rooms = re.search("\d+", string).group()
        room_type = re.search("[a-z]+\D*", string).group()
        room_data[room_type] = number_of_rooms
    
    
    # Add missing room types with value 0
    for room_type in room_types:
        if room_type not in room_data.index:
            room_data[room_type] = 0

    # Consistent column order
    return room_data[room_types]

In [750]:
# Extract data from "Main unit"
missing_room_data = main_split.\
    apply(lambda rows: extract_rooms(rows)).\
        astype('int')
missing_room_data

/home/jahnic/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,rooms,room,bedrooms,bedroom,bathrooms,bathroom,powder rooms,powder room
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
5419,0,0,0,0,0,0,0,0
5420,0,0,0,0,0,0,0,0
5421,0,0,0,0,0,0,0,0
5422,4,0,2,0,0,1,0,0


In [751]:
# Merge related columns
rooms = missing_room_data.rooms + missing_room_data.room
bedrooms = missing_room_data.bedrooms + missing_room_data.bedroom
bathrooms = missing_room_data.bathrooms + missing_room_data.bathroom
powder_rooms = missing_room_data['powder rooms'] + missing_room_data['powder room']

In [752]:
print("Rooms before missing data suplementation")
df.iloc[:,-5:].apply(lambda cols: sum(not_zero(cols)))

Rooms before missing data suplementation


rooms               4451
bedrooms            4402
basement_bedroom      75
bathrooms           4432
powder_rooms        1167
dtype: int64

In [753]:
df.rooms = df.rooms + rooms
df.bedrooms = df.bedrooms + bedrooms
df.bathrooms = df.bathrooms + bathrooms
df['powder_rooms'] = df['powder_rooms'] + powder_rooms

In [796]:
print("Rooms after missing data suplementation")
df.iloc[:,-5:].apply(lambda cols: sum(not_zero(cols)))

Rooms after missing data suplementation


rooms               5319
bedrooms            5264
basement_bedroom      75
bathrooms           5296
powder_rooms        1300
dtype: int64

In [798]:
df.drop("Main unit", axis=1, inplace=True)

Lastly, we will inspect the missing `rooms` values to see if we could substitute them from other room related columns.

In [772]:
# Slice with "rooms" values 0
no_rooms = df[df.rooms.apply(lambda x: x == 0)]
no_rooms_slice = no_rooms.loc[:, ["rooms", "bedrooms", "bathrooms", "powder_rooms"]]
print(no_rooms_slice, "\n")

# Search non-zero values in slice
print("Non-zero values in slice:",\
no_rooms_slice.apply(lambda row: any([x > 0 for x in row])).sum())

rooms  bedrooms  bathrooms  powder_rooms
154     0.0       0.0        0.0           0.0
164     0.0       0.0        0.0           0.0
344     0.0       0.0        0.0           0.0
345     0.0       0.0        0.0           0.0
532     0.0       0.0        0.0           0.0
...     ...       ...        ...           ...
5381    0.0       0.0        0.0           0.0
5394    0.0       0.0        0.0           0.0
5415    0.0       0.0        0.0           0.0
5416    0.0       0.0        0.0           0.0
5423    0.0       0.0        0.0           0.0

[105 rows x 4 columns] 

Non-zero values in slice: 0


There are no values to further substitute the missing room information

In [773]:
no_rooms.title

154     Land for sale
164      Lot for sale
344      Lot for sale
345      Lot for sale
532      Lot for sale
            ...      
5381     Lot for sale
5394     Lot for sale
5415     Lot for sale
5416     Lot for sale
5423     Lot for sale
Name: title, Length: 105, dtype: object

The missing room values appear to correspond to lots/land for sale. We will delete these records.

In [780]:
df = df.drop(no_rooms.index).reset_index()

In [781]:
df.info(max_cols=10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5319 entries, 0 to 5318
Columns: 91 entries, index to powder_rooms
dtypes: bool(1), float64(26), int64(40), object(24)
memory usage: 3.7+ MB


## Columns With Many Missing Values

To facilitate working with column names we will transform them to lower-case and replace whitespace with underscores

In [804]:
df.columns = df.columns.str.replace(" ", "_").str.lower()

In [805]:
# Columns with less than 1000 values
less_than_3100 = df.apply(lambda col: col.notnull().sum() < 3100, axis=0)
df.loc[:, less_than_3100].head()

,net_area,pool,additional_features,gross_area,building_style,lot_area,fireplace/stove,in_basement,use_of_property,number_of_units,residential_units,potential_gross_revenue,intergenerational,building_area_(at_ground_level),available_area,residential_unit,parking_(excluded_from_the_price)
0,"1,014 sqft",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,249 sqft","Heated, Inground",Elevator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,848 sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"Heated, Inground, Indoor","Adapted for reduced mobility, Elevator",621 sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,586 sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [795]:
# Columns with more than 3100 values
df.loc[:, [not many_missing for many_missing in less_than_3100]].info(max_cols=75)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5319 entries, 0 to 5318
Data columns (total 72 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   index                                    5319 non-null   int64  
 1   title                                    5319 non-null   object 
 2   address                                  5319 non-null   object 
 3   price                                    5319 non-null   int64  
 4   lat                                      5319 non-null   float64
 5   long                                     5319 non-null   float64
 6   Restaurants                              5311 non-null   float64
 7   Shopping                                 5313 non-null   float64
 8   Vibrant                                  5311 non-null   float64
 9   Cycling friendly                         5310 non-null   float64
 10  Car friendly                             5308 no

Since we deleted the lots for sale, we will first remove the now redundant `Zoning` feature

In [800]:
df.drop("Zoning", axis=1, inplace=True)

Next we will inspect the `area` features and see if we can merge them together.

In [807]:
area = ["net_area", "gross_area", "building_area_(at_ground_level)", "lot_area", "available_area"]
df_area = df[area]
df_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5319 entries, 0 to 5318
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   net_area                         2616 non-null   object
 1   gross_area                       654 non-null    object
 2   building_area_(at_ground_level)  205 non-null    object
 3   lot_area                         1696 non-null   object
 4   available_area                   110 non-null    object
dtypes: object(5)
memory usage: 207.9+ KB


`Net area` and `Gross area` seem like the same thing, so first we will inspect if there is any intersection between both features

In [889]:
net_not_missing = df_area.net_area.notna()
gross_not_missing = df_area.gross_area.notna()
overlap = (net_not_missing & gross_not_missing)
print("Net and Gross area records overlap:", overlap.sum())

Net and Gross area records overlap: 0


There is no overlap between the two which means it should be reasonable to combine both features. Before we can add both features together, we need to convert values from string to numeric.

In [854]:
# Remove 'sqft' and ',' from area features
df_area.replace(regex="sqft", value="", inplace=True)
df_area.replace(regex=",", value="", inplace=True)
df_area.rename(columns={"building_area_(at_ground_level)":"building_area"}, inplace=True)
df_area

,net_area,gross_area,building_area,lot_area,available_area
0,1014,NaN,NaN,NaN,NaN
1,1249,NaN,NaN,NaN,NaN
2,848,NaN,NaN,NaN,NaN
3,NaN,621,NaN,NaN,NaN
4,586,NaN,NaN,NaN,NaN
...,...,...,...,...,...
5314,552,NaN,NaN,NaN,NaN
5315,808,NaN,NaN,NaN,NaN
5316,1390,NaN,NaN,NaN,NaN
5317,1218,NaN,NaN,NaN,NaN


In [868]:
# Merge "net_area" and "gross_area" -> "total_area"
total_area = df_area.net_area.astype(float).fillna(0)\
    + df_area.gross_area.astype(float).fillna(0)

df["total_area"] = total_area
missing = total_area.apply(lambda x: x == 0)
print("Remaining missing values after merge of 'net' and 'gross':", missing.sum())

Remaining missing values after merge of 'net' and 'gross': 2049


Next we will inspect if there's any overlap for the newly merged column and `lot_area`

In [880]:
not_missing_total = [not x for x in missing]
has_lot_area = df_area.lot_area.notna()
overlap =  not_missing_total & has_lot_area 
print("Total and lot area records overlap:", overlap.sum())

Total and lot area records overlap: 0


Again, no overlap between columns which suggests, that total and lot area are the same. Listings that indicate lot area, most likely include property outside that of the building, such as a backyard. This has been verified by scrolling through pictures of listings with lot areas. This information may be of value, which is why we will create a boolean column `has_lot_area` to indicate listings with lot areas.

In [882]:
df['has_lot_area'] = has_lot_area

Next we will need to convert the lot_area column to float

In [891]:
df_area.lot_area

5        3030
7        1827
8        4090
14       3174
20       2584
        ...  
5307    12002
5309    12400
5311     3138
5312     2717
5318     2990
Name: lot_area, Length: 1696, dtype: object